In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
import os

In [2]:
from tsai.all import *
import sklearn.metrics as skm
my_setup()

os              : Windows-10-10.0.19045-SP0
python          : 3.9.16
tsai            : 0.3.9
fastai          : 2.7.15
fastcore        : 1.5.54
torch           : 2.0.1
device          : 1 gpu (['NVIDIA GeForce GTX 1050 Ti'])
cpu cores       : 4
threads per cpu : 2
RAM             : 7.88 GB
GPU memory      : [4.0] GB


In [3]:
# 导入数据集
X = np.load('./dataset2/X_train.npy')
Y = pd.read_csv('./dataset2/Y_train.csv')
X_val = np.load('./dataset2/X_test.npy')
submits = pd.read_csv('./submit/submit2.csv')

In [4]:
# 载入数据
fake_y = np.random.randint(0, 3, size=X_val.shape[0])
X_val, fake_y,_ = combine_split_data([X_val], [fake_y])

In [9]:
# 带入训练数据并预测结果
result_dict = []
for fold_num in range(5):
    # 训练模型储存路径
    path = './question2-tsai-result/original/XceptionTimePlus/fold_{}'.format(fold_num)
    # 训练模型载入
    learn = load_all(path=path, dls_fname='original', model_fname='XceptionTimePlus', learner_fname='learner')
    # 预测结果
    probas, targets = learn.get_X_preds(X_val, with_decoded=False)
    preds = probas.argmax(axis=1).cpu().numpy().tolist()
    result_dict.append(preds)

# 用投票法预测最终的分类
all_predictions = np.array(result_dict)
# 计算每个类别的投票数
votes = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), 0, all_predictions)
# 输出
submits['Label'] = votes+1
os.makedirs('./results/XceptionTimePlus', exist_ok=True)
submits.to_csv('./results/XceptionTimePlus/submit2.csv', index=False)

In [10]:
submits.head(10)

,Id,Label
0,1,3
1,2,2
2,3,3
3,4,2
4,5,3
5,6,3
6,7,3
7,8,3
8,9,2
9,10,1
